In [ ]:
# 필수 패키지 설치
import sys
import os
sys.path.append(os.path.abspath('../backend'))

# 기본 패키지
import openai
import sqlite3
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Tuple
import json
import re
import time

# 실험용 추가 패키지 (필요시 설치)
try:
    from sentence_transformers import SentenceTransformer
    from sklearn.metrics.pairwise import cosine_similarity
    from fuzzywuzzy import fuzz
    ADVANCED_LIBS_AVAILABLE = True
except ImportError:
    print("고급 라이브러리들이 설치되지 않음. 설치하려면:")
    print("pip install sentence-transformers scikit-learn fuzzywuzzy python-levenshtein")
    ADVANCED_LIBS_AVAILABLE = False

print("환경설정 완료!")


In [ ]:
class ExperimentalLLMNormalizer:
    """
    실험용 LLM 기반 정규화 엔진
    - 실제 프로젝트: backend/app/logic/normalizer.py
    - 개선된 로직을 실제 normalizer.py에 복사/치환하여 반영
    
    실험 가능한 요소들:
    1. LLM 모델 변경 (GPT-4, GPT-3.5, HuggingFace 등)
    2. 프롬프트 엔지니어링
    3. 표준 용어 사전 관리 방식
    4. 유사도 계산 알고리즘
    """
    
    def __init__(self, api_key: str, model: str = "gpt-3.5-turbo"):
        """
        실험용 정규화 엔진 초기화
        
        Args:
            api_key: OpenAI API 키
            model: 사용할 LLM 모델 (실험 시 교체 가능)
        """
        openai.api_key = api_key
        self.model = model
        
        # 표준 용어 사전 (실험 시 수정/확장 가능)
        self.standard_terms = {
            "equipment": [
                "[VEDR]Pressure Vessel/ Drum", "[GVGV]Guided Vehicle/ Guided Vehicle", 
                "[ANAA]Analyzer/ Analyzer", "[MVVV]Motor Operated Valve/ Motor Operated Valve",
                "[CONV]Conveyor/ Conveyor", "[PUMP]Pump/ Pump", "[HEXH]Heat Exchanger/ Heat Exchanger",
                "[VALV]Valve/ Valve", "[CTLV]Control Valve/ Control Valve", "[TANK]Tank/ Tank",
                "[STNK]Storage Tank/ Storage Tank", "[FILT]Filter/ Filter", "[REAC]Reactor/ Reactor",
                "[COMP]Compressor/ Compressor", "[FAN]Fan/ Fan", "[BLOW]Blower/ Blower"
            ],
            "location": [
                "No.1 PE", "No.2 PE", "Nexlene 포장 공정", "PW-B3503", "SKGC-설비관리", 
                "KNC-설비관리", "석유제품배합/저장", "합성수지 포장", "RFCC", "1창고 #7Line", "2창고 #8Line", "공통 시설"
            ],
            "status": [
                "고장", "누설", "작동불량", "소음", "진동", "온도상승", 
                "압력상승", "주기적 점검/정비", "고장.결함.수명소진", "SHE", "운전 Condition 이상"
            ]
        }
    
    def normalize_term(self, term: str, category: str) -> Tuple[str, float]:
        """
        LLM을 사용하여 용어를 표준 용어로 정규화 (실험용)
        
        Args:
            term: 정규화할 용어 (예: "압력베젤", "모터밸브")
            category: 용어 카테고리 ("equipment", "location", "status")
            
        Returns:
            (표준용어, 신뢰도점수): 정규화된 표준 용어와 신뢰도 (0.0~1.0)
        """
        if not term:
            return term, 0.0
        
        try:
            # 프롬프트 생성 (실험 시 개선 가능)
            prompt = self._create_normalization_prompt(term, category)
            
            # LLM 호출 (실험 시 모델/파라미터 변경 가능)
            response = openai.ChatCompletion.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "당신은 설비관리 시스템의 용어 정규화 전문가입니다."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1,  # 실험 시 조정 가능
                max_tokens=200
            )
            
            result_text = response.choices[0].message.content.strip()
            
            # 응답 파싱 (실험 시 개선 가능)
            normalized_term, confidence = self._parse_normalization_response(result_text)
            
            return normalized_term, confidence
            
        except Exception as e:
            print(f"LLM 정규화 오류: {e}")
            return term, 0.5  # 오류 시 원본 반환, 중간 신뢰도

print("ExperimentalLLMNormalizer 클래스 정의 완료")


In [ ]:
class RuleBasedNormalizer:
    """
    규칙 기반 정규화 엔진 (실험용)
    - LLM 비용 절약 및 빠른 응답속도
    - 정확한 매핑 규칙 기반
    - 확장 가능한 규칙 사전
    """
    
    def __init__(self):
        # 매핑 규칙 사전 (실험 시 확장 가능)
        self.equipment_mapping = {
            "압력베젤": "Pressure Vessel",
            "베젤": "Pressure Vessel", 
            "압력용기": "Pressure Vessel",
            "모터밸브": "Motor Operated Valve",
            "밸브": "Valve",
            "펌프": "Pump",
            "컨베이어": "Conveyor",
            "탱크": "Tank",
            "저장탱크": "Storage Tank"
        }
        
        self.location_mapping = {
            "1PE": "No.1 PE",
            "2PE": "No.2 PE", 
            "석유제품": "석유제품배합/저장",
            "합성수지": "합성수지 포장"
        }
        
        self.status_mapping = {
            "고장": "고장",
            "누설": "누설", 
            "작동불량": "작동불량",
            "이상": "작동불량"
        }
    
    def normalize_term(self, term: str, category: str) -> Tuple[str, float]:
        """규칙 기반 정규화"""
        if not term:
            return term, 0.0
        
        # 정확 매칭 시도
        mapping_dict = getattr(self, f"{category}_mapping", {})
        
        # 정확 매칭
        if term in mapping_dict:
            return mapping_dict[term], 1.0
        
        # 부분 매칭 시도
        for key, value in mapping_dict.items():
            if key in term or term in key:
                return value, 0.8
        
        # 유사도 기반 매칭 (fuzzywuzzy 사용)
        if ADVANCED_LIBS_AVAILABLE:
            best_match = None
            best_score = 0
            
            for key, value in mapping_dict.items():
                score = fuzz.ratio(term, key) / 100.0
                if score > best_score and score > 0.6:
                    best_score = score
                    best_match = value
            
            if best_match:
                return best_match, best_score
        
        return term, 0.3  # 매칭 실패시 원본 반환

# 실험 예시
def test_normalizers():
    """정규화 엔진들 비교 테스트"""
    # API 키 설정 (실제 실험시 입력)
    OPENAI_API_KEY = "your-api-key-here"
    
    test_terms = [
        ("압력베젤", "equipment"),
        ("모터밸브", "equipment"), 
        ("1PE", "location"),
        ("고장", "status")
    ]
    
    if OPENAI_API_KEY != "your-api-key-here":
        llm_normalizer = ExperimentalLLMNormalizer(OPENAI_API_KEY)
    rule_normalizer = RuleBasedNormalizer()
    
    print("=== 정규화 엔진 비교 테스트 ===")
    for term, category in test_terms:
        print(f"\\n입력: {term} ({category})")
        
        # Rule-based 결과
        rule_result, rule_conf = rule_normalizer.normalize_term(term, category)
        print(f"Rule-based: {rule_result} (신뢰도: {rule_conf:.2f})")
        
        # LLM 결과 (API 키가 있는 경우)
        if OPENAI_API_KEY != "your-api-key-here":
            try:
                llm_result, llm_conf = llm_normalizer.normalize_term(term, category)
                print(f"LLM-based: {llm_result} (신뢰도: {llm_conf:.2f})")
            except Exception as e:
                print(f"LLM 오류: {e}")

print("Rule-based 정규화 엔진 및 테스트 함수 준비 완료!")
